In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
import sys

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.readdata import *

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [6]:
data_date = "2018_05_29_13_52_44"
data_dir = "/home/pleroy/DATA/PoSAR-v2_PIMA_TEST-3/2018_05_29/" + data_date

In [7]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )
params.print()

requestedTRamp 1200.0
configuredTRamp 1200.0
startFrequency 5725000000.0
stopFrequency 5875000000.0
frequencyBand 150000000.0
frequencyDevReq 50000.0
frequencyDevConf 50001.1
numberOfSteps 3000
waveformType 1
rampsPerBuffer 375
bufferSize 9000000.0
buffersPerFile 2
rampsPerFile 750
fileSize 18000000.0
samplingFrequency 10000000.0
samplesPerRamp 12000
skipNSamples 0


In [8]:
buffersPerFile = params.buffersPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "buffersPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    buffersPerFile, samplesPerRamp, rampsPerFile )
     )

# selection in record
firstFile = 90
nbFiles = 73
lastFile = firstFile + nbFiles - 1
firstBuffer = firstFile * buffersPerFile

buffersPerFile = 2, samplesPerRamp = 12000, rampsPerFile = 750


# Read the data

In [9]:
scalingFactor = 2 / 65535
offset = -32768

In [10]:
timeSerie_A = np.zeros( samplesPerFile )
adc_A = np.zeros( (nbFiles, samplesPerFile) )

# Read all files

In [11]:
lastBuffer = (nbFiles-1) * buffersPerFile + firstBuffer
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * buffersPerFile + firstBuffer) )
    filename = data_dir + "/record" + nb + ".bin"
    readFileADLINKCh0( filename, samplesPerFile, timeSerie_A )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBuffer))
    
    adc_A[ k, : ] = (timeSerie_A + offset) * scalingFactor

k = 0, block = 180 / 324
k = 1, block = 182 / 324
k = 2, block = 184 / 324
k = 3, block = 186 / 324
k = 4, block = 188 / 324
k = 5, block = 190 / 324
k = 6, block = 192 / 324
k = 7, block = 194 / 324
k = 8, block = 196 / 324
k = 9, block = 198 / 324
k = 10, block = 200 / 324
k = 11, block = 202 / 324
k = 12, block = 204 / 324
k = 13, block = 206 / 324
k = 14, block = 208 / 324
k = 15, block = 210 / 324
k = 16, block = 212 / 324
k = 17, block = 214 / 324
k = 18, block = 216 / 324
k = 19, block = 218 / 324
k = 20, block = 220 / 324
k = 21, block = 222 / 324
k = 22, block = 224 / 324
k = 23, block = 226 / 324
k = 24, block = 228 / 324
k = 25, block = 230 / 324
k = 26, block = 232 / 324
k = 27, block = 234 / 324
k = 28, block = 236 / 324
k = 29, block = 238 / 324
k = 30, block = 240 / 324
k = 31, block = 242 / 324
k = 32, block = 244 / 324
k = 33, block = 246 / 324
k = 34, block = 248 / 324
k = 35, block = 250 / 324
k = 36, block = 252 / 324
k = 37, block = 254 / 324
k = 38, block = 256 / 

In [12]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)

**Check that the first ramp is an up ramp**

In [13]:
plt.figure()
plt.plot( A_reshaped[0, :] )
plt.title("first acquisition\n" + data_date)
plt.grid()

**Set the shifted flag properly**

In [14]:
rampDownFirst = 1

## Spectrum

### No window

In [15]:
samplesPerDownRamp = int(samplesPerRamp/2)

In [16]:
#ifft = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp], axis=1)

In [17]:
#ifft_abs = 20 * np.log10( np.abs( ifft ) )
ifft_abs = 20 * np.log10( np.abs( np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp], axis=1) ) )

In [43]:
plt.figure()
plt.imshow( ifft_abs[::100, 300:3000] )
title = "no window - ifft_abs - " + data_date
plt.title(title)
plt.savefig( data_dir + "/" + title + ".png" )

In [62]:
shift = 8
nbPlots = 100
plt.figure()
for k in range(nbPlots):
    plt.plot(ifft_abs[ k + shift * nbPlots, 0:500 ])
plt.title("shift = {}, nbPlots = {}".format(shift, nbPlots))
plt.grid()

In [67]:
shift = 5
nbPlots = 100
plt.figure()
for k in range(nbPlots):
    plt.plot(A_reshaped[ k + shift * nbPlots, : ])
plt.title("shift = {}, nbPlots = {}".format(shift, nbPlots))
plt.grid()

In [34]:
#fft = np.fft.fft(A_reshaped[:, 0:samplesPerDownRamp], axis=1)

In [35]:
#fft_abs = 20 * np.log10( np.abs( fft ) )

In [36]:
#plt.figure()
#plt.imshow( fft_abs[::100, 0:1000] )
#title = "no window - fft_abs - " + data_date
#plt.title(title)
#plt.savefig( data_dir + "/" + title + ".png" )

### Hamming

In [ ]:
hamming = np.hamming(samplesPerDownRamp)

In [ ]:
fft_A_hamming = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*hamming, axis=1)

In [ ]:
fft_abs_hamming = 20 * np.log10( np.abs( fft_A_hamming ) )

In [ ]:
plt.figure()
plt.imshow( fft_abs_hamming[::100, 0:1000], cmap='gray' )
title = "hamming " + data_date
plt.title(title)
plt.savefig( data_dir + "/" + title + ".png" )

### Hanning

In [ ]:
hanning = np.hanning(samplesPerDownRamp)

In [ ]:
fft_A_hanning = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*hanning, axis=1)

In [ ]:
fft_abs_hanning = 20 * np.log10( np.abs( fft_A_hanning ) )

In [ ]:
plt.figure()
plt.imshow( fft_abs_hanning[::100, 0:1000], cmap='gray' )
title = "hanning " + data_date
plt.title(title)
plt.savefig( data_dir + "/" + title + ".png" )

### Blackman (peu utilisée, cf lff)

In [ ]:
blackman = np.blackman(samplesPerDownRamp)

In [ ]:
fft_A_blackman = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*blackman, axis=1)

In [ ]:
fft_abs_blackman = 20 * np.log10( np.abs( fft_A_blackman ) )

In [ ]:
plt.figure()
plt.imshow( fft_abs_blackman[::100, 0:1000], cmap='gray' )
title = "blackman " + data_date
plt.title(title)
plt.savefig( data_dir + "/" + title + ".png" )

### Kaiser

In [ ]:
# beta Window shape
# 0    Rectangular
# 5    Similar to a Hamming
# 6    Similar to a Hanning
# 8.6  Similar to a Blackman
# kaiser = np.kaiser(samplesPerDownRamp)

In [ ]:
fft_A_kaiser = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*kaiser, axis=1)

In [ ]:
fft_abs_kaiser = 20 * np.log10( np.abs( fft_A_kaiser ) )

In [ ]:
plt.figure()
plt.imshow( fft_abs_kaiser[::100, 0:1000], cmap='gray' )
plt.title("kaiser " + data_date)

# Build RD

In [ ]:
#RDc = build_rd_from_data( params, A_reshaped, shifted )
withHanning = 0
RDc = build_rd_from_data_rampDown( params, A_reshaped, rampDownFirst, withHanning )

In [ ]:
coupling_RD = np.average(RDc, 0)
if withHanning:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown_hanning".format(firstFile, lastFile), coupling_RD )
else:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown".format(firstFile, lastFile), coupling_RD )

In [ ]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [ ]:
if withHanning:
    np.save(data_dir + '/RD_files_{}_{}_rampDown_hanning'.format(firstFile, lastFile), RDc) 
else:
    np.save(data_dir + '/RD_files_{}_{}_rampDown'.format(firstFile, lastFile), RDc)